In [13]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
import re

In [14]:
all_years = [2022, 2021, 2020, 2019, 2018]
all_team_data = []
scrape_url = "https://fbref.com/en/comps/9/Premier-League-Stats"
for curr_year in all_years:
    html_text = requests.get(scrape_url)
    bsObj = BeautifulSoup(html_text.text)
    standings_table = bsObj.find_all("table", {"id":re.compile("results.*")})[0]
    all_links_list = [link.get("href") for link in standings_table.find_all('a')]
    all_links_list = [link for link in all_links_list if '/squads/' in link]
    team_urls = [f"https://fbref.com{link}" for link in all_links_list]
    
    past_season = bsObj.select("a.prev")[0].get("href")
    scrape_url = f"https://fbref.com{past_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1]
        team_name = team_name.replace("-Stats", "").replace("-", " ")
        html_text = requests.get(team_url)
        matches = pd.read_html(html_text.text, match="Scores & Fixtures")[0]
        bsObj = BeautifulSoup(html_text.text)
        all_links_list = [link.get("href") for link in bsObj.find_all('a')]
        all_links_list = [link for link in all_links_list if link and 'all_comps/shooting/' in link]
        html_text = requests.get(f"https://fbref.com{all_links_list[0]}")
        shooting_data = pd.read_html(html_text.text, match="Shooting")[0]
        shooting_data.columns = shooting_data.columns.droplevel()
        try:
            curr_team = matches.merge(shooting_data[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
        curr_team = curr_team[curr_team["Comp"] == "Premier League"]
        curr_team["Season"] = curr_year
        curr_team["Team"] = team_name
        all_team_data.append(curr_team)
        time.sleep(1) #to avoid getting blocked for scrapping too fast

In [15]:
matches_df = pd.concat(all_team_data)
matches_df.columns = [c.lower() for c in matches_df.columns]
matches_df.head()

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
0,2022-08-05,20:00,Premier League,Matchweek 1,Fri,Away,W,2.0,0.0,Crystal Palace,...,Match Report,NaN,10.0,2.0,14.6,1.0,0.0,0.0,2022,Arsenal
1,2022-08-13,15:00,Premier League,Matchweek 2,Sat,Home,W,4.0,2.0,Leicester City,...,Match Report,NaN,19.0,7.0,13.0,0.0,0.0,0.0,2022,Arsenal
2,2022-08-20,17:30,Premier League,Matchweek 3,Sat,Away,W,3.0,0.0,Bournemouth,...,Match Report,NaN,14.0,6.0,14.8,0.0,0.0,0.0,2022,Arsenal
3,2022-08-27,17:30,Premier League,Matchweek 4,Sat,Home,W,2.0,1.0,Fulham,...,Match Report,NaN,22.0,8.0,15.5,1.0,0.0,0.0,2022,Arsenal
4,2022-08-31,19:30,Premier League,Matchweek 5,Wed,Home,W,2.0,1.0,Aston Villa,...,Match Report,NaN,22.0,8.0,16.3,1.0,0.0,0.0,2022,Arsenal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35,2019-04-13,12:30,Premier League,Matchweek 34,Sat,Away,L,0,4,Tottenham,...,Match Report,NaN,7.0,1.0,18.9,1.0,0.0,0.0,2018,Huddersfield Town
36,2019-04-20,15:00,Premier League,Matchweek 35,Sat,Home,L,1,2,Watford,...,Match Report,NaN,13.0,3.0,18.1,1.0,0.0,0.0,2018,Huddersfield Town
37,2019-04-26,20:00,Premier League,Matchweek 36,Fri,Away,L,0,5,Liverpool,...,Match Report,NaN,5.0,1.0,21.6,0.0,0.0,0.0,2018,Huddersfield Town
38,2019-05-05,14:00,Premier League,Matchweek 37,Sun,Home,D,1,1,Manchester Utd,...,Match Report,NaN,7.0,3.0,19.0,1.0,0.0,0.0,2018,Huddersfield Town


In [16]:
matches_df.to_csv("epl.csv")